<a href="https://colab.research.google.com/github/kxk302/MBA/blob/main/MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls '/content/gdrive/MyDrive/Colab Notebooks/MBA_files'

In [120]:
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

funclass_dict = {"1":"Silent", "2":"Nonsense", "3":"Misense", "4":"None"}
af_dict = {"1":"AF < 0.20", "2": "0.80 > AF >= 0.20", "3":"AF >= 0.80"}

def to_str_each(items):
  items_str = ""
  for item in items:
    item_str = "(" + item[0:-2:1] + ", " + funclass_dict.get(item[-2:-1:1]) + ", " + af_dict.get(item[-1]) + ") & "
    items_str += item_str 
  return items_str[:-3]

# Convert series of antecedents/consequents to human readable 
def to_str(ser):    
  # Cast to string
  ser = ser.astype(str)  

  # Get rid of unnecessary characters prior to list of (position + funclass + af) numbers
  ser = ser.str.slice(12,-3,1)  

  # Get rid of single quotes
  ser = ser.replace('\'', '', regex=True)

  ser = ser.str.split(pat=",")

  return ser.apply(to_str_each)

def add_readable_rules(df_in):
  # Empty data frame
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  antecedents_str = to_str(df['antecedents'])
  consequents_str = to_str(df['consequents'])

  readable_rule = "(" + antecedents_str + ") => (" + consequents_str + ")"
  df.insert(2,'readable_rule', readable_rule)

  return df

def filter_on_position_probability(df_in, start_prob=None, end_prob=None):
    if df_in is None or df_in.shape[0] == 0:
      return df_in

    if start_prob is None and end_prob is None:
      return df_in

    # Only consider rows where probability of Position being present in the Samples is >= start_prob and <= end_prob
    num_samples = df_in['Sample'].nunique()
    print("num_samples: {}".format(num_samples))

    num_positions = df_in['POS'].nunique()
    print("num_positions: {}".format(num_positions))
   
    df = df_in[['Sample', 'POS']].groupby(['POS']).agg(position_prob=('Sample', 'count'))
    # Normalize position_prob by dividing it by num_samples. This makes it a proability between 0.0 and 1.0
    df['position_prob'] = df['position_prob'] / num_samples   

    df_sorted = df.sort_values(by = 'position_prob')
    print(df_sorted.head())
    print(df_sorted.tail())
  
    if start_prob is not None:
      df = df[df.position_prob >= start_prob]

    if end_prob is not None:
      df = df[df.position_prob <= end_prob]
    
    ret_val = pd.merge(df_in, df, on='POS')

    df_sorted = ret_val.sort_values(by = 'position_prob')[['POS', 'position_prob']]
    print(df_sorted.head())
    print(df_sorted.tail())

    return ret_val

def get_position_probability(df_in, position=None):
    if df_in is None or df_in.shape[0] == 0:
      return df_in

    if position is None:
      return df_in

    num_samples = df_in['Sample'].nunique()
    print("num_samples: {}".format(num_samples))

    num_positions = df_in['POS'].nunique()
    print("num_positions: {}".format(num_positions))
   
    df = df_in[['Sample', 'POS']].groupby(['POS']).agg(position_prob=('Sample', 'count'))
    # Normalize position_prob by dividing it by num_samples. This makes it a proability between 0.0 and 1.0
    df['position_prob'] = df['position_prob'] / num_samples

    try:      
      return df.loc[position,'position_prob']
    except KeyError:
      return -1

def filter_on_column_values(df_in, start_pos=None, end_pos=None, start_af=None, end_af=None, funclass=[]):  
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  # Replace "." with "NONE" in FUNCLASS column. They both represent "Non-coding" variant
  df["FUNCLASS"] = df["FUNCLASS"].replace('.', 'NONE')

  # Filter in_file rows
  if start_pos is not None:
    df = df[df.POS >= start_pos]    
  if end_pos is not None:
    df = df[df.POS <= end_pos]    
  if start_af is not None:
    df = df[df.AF >= start_af]    
  if end_af is not None:
    df = df[df.AF <= end_af]    
  if len(funclass) > 0:    
    df = df[df.FUNCLASS.isin(funclass)]

  return df

# Create a single integer representing a variant at a specific position with a specific allele frequency
# Pivot the data so we have all sample variants on a single line
#
# Extract rows from in_file where
# 
#    POS >= start_pos & POS <= end_pos
#        If start_pos = None: POS <= end_pos
#        If end_pos = None: POS >= start_pos 
#
#    AF >= start_af & AF <= end_af
#        If start_af = None: AF <= end_af
#        If end_af = None: AF >= start_af
#
#    FUNCLASS in funclass (funclass is a list)      
#
def preprocess_input_file(df_in):
  if df_in is None or df_in.shape[0] == 0:
    return df_in
  
  df = df_in.copy()

  # Bucket values in FUNCLASS and AF columns. We do not bucket the values in POS column.

  # Replace variants in FUNCLASS column with a distinct numeric value
  df.loc[df.FUNCLASS == "NONE", "FUNCLASS"] = 4
  df.loc[df.FUNCLASS == "MISSENSE", "FUNCLASS"] = 3
  df.loc[df.FUNCLASS == "NONSENSE", "FUNCLASS"] = 2 
  df.loc[df.FUNCLASS == "SILENT", "FUNCLASS"] = 1 

  # Replace allele frequency in AF column with a distict numeric value
  df.loc[df.AF >= 0.80, "AF"] = 3
  df.loc[(df.AF >= 0.20) & (df.AF < 0.80), "AF"] = 2
  df.loc[df.AF < 0.20, "AF"] = 1

  # Convert AF values to integer
  df = df.astype({"AF": int}) 

  # Create a new column called 'Label', which is a string concatentation of POS, FUNCLASS, and AF values. 
  # The idea is to represent each variant + allele frequency + position as a single integer, to be used in MBA 
  df["Label"] = df["POS"].astype(str) + df["FUNCLASS"].astype(str) + df["AF"].astype(str)

  # We do not need POS, FUNCLASS, and AF columns anymore
  df = df[["Sample", "Label"]]
  
  # Add a new column called 'Value', prepopulated with 1
  df["Value"] = 1

  df = pd.pivot_table(df, index="Sample", columns="Label", values="Value")

  # Set all data frame nan (not a number) values to 0
  df = df.fillna(0)

  # Convert all data framevalues to integer
  df = df.astype(int) 

  return df

In [112]:
def get_association_rules(in_file, min_support=0.20, min_confidence=0.80, min_lift=1.0, min_conviction=1.0, max_len=None, start_pos=None, end_pos=None, start_af=None, end_af=None, funclass=[], start_prob=None, end_prob=None):
  # Read the input file and pick the needed columns
  df_in = pd.read_csv(in_file, sep='\t')[['Sample', 'POS', 'AF', 'FUNCLASS']]

  # Filter on position probability
  df_pp = filter_on_position_probability(df_in, start_prob, end_prob)

  # Filter on column values
  df_cv = filter_on_column_values(df_pp, start_pos, end_pos, start_af, end_af, funclass)

  # Preprocess the data frame
  df = preprocess_input_file(df_cv)

  # Get frequent item sets, with support larger than min_support, using Apriori algorithm
  frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True, max_len=max_len)

  # Get association rules, with lift larger than min_lift  
  rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)

  # Filter association rules, keeping rules with confidence larger than min_confidence
  rules = rules[ (rules['confidence'] >= min_confidence) & (rules['conviction'] >= min_conviction) ]

  return rules

In [ ]:
pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

bos_rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz", min_support=0.223, min_confidence=0.905, min_lift=2.863, min_conviction=7.0)
num_rules = bos_rules.shape[0]
print('Boston dataset association rules: ')
bos_rules = add_readable_rules(bos_rules)
print(bos_rules.head(num_rules))
print('\n\n')
# bos_rules.to_csv('/content/gdrive/MyDrive/Colab Notebooks/MBA_files/bos_0223_0905_2863_7000.csv', sep=',')

uke_rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20200917_by_sample.tsv.gz", min_support=0.21, min_confidence=0.91, min_lift=2.5, min_conviction=7.0)
num_rules = uke_rules.shape[0]
print('UK early dataset association rules: ')
uke_rules = add_readable_rules(uke_rules)
print(uke_rules.head(num_rules))
print('\n\n')
# uke_rules.to_csv('/content/gdrive/MyDrive/Colab Notebooks/MBA_files/uke_0210_0910_2500_7000.csv', sep=',')

ukl_rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20201120_by_sample.tsv.gz", min_support=0.203, min_confidence=0.926, min_lift=2.03, min_conviction=7.0)
num_rules = ukl_rules.shape[0]
print('Uk late dataset association rules: ')
ukl_rules = add_readable_rules(ukl_rules)
print(ukl_rules.head(num_rules))
print('\n\n')
# ukl_rules.to_csv('/content/gdrive/MyDrive/Colab Notebooks/MBA_files/ukl_0203_0926_2030_7000.csv', sep=',')


In [ ]:
# Milad 
pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

bos_rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz", min_support=0.005, min_confidence=0.05, min_lift=1.00, min_conviction=1.0, max_len=2, start_pos=21563, end_pos=25384, start_af=0.19, end_af=None, funclass=["MISSENSE", "NONSENSE"])
num_rules = bos_rules.shape[0]
print('Boston dataset association rules: ')
bos_rules = add_readable_rules(bos_rules)
print(bos_rules.head(num_rules))
print('\n\n')
# bos_rules.to_csv('/content/gdrive/MyDrive/Colab Notebooks/MBA_files/bos_0223_0905_2863_7000.csv', sep=',')


In [124]:
# Anton
pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

bos_rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz", min_support=0.050, min_confidence=0.800, min_lift=1.0, min_conviction=1.0, max_len=2, end_af=0.80, start_prob=0.000, end_prob=0.40)
num_rules = bos_rules.shape[0]
print('Number of rules: {}'.format(num_rules))
print('Boston dataset association rules: ')
bos_rules = add_readable_rules(bos_rules)
print(bos_rules.head(num_rules))
print('\n\n')


num_samples: 639
num_positions: 1013
       position_prob
POS                 
28893       0.001565
18546       0.001565
28647       0.001565
18636       0.001565
18714       0.001565
       position_prob
POS                 
25563       0.852895
241         0.948357
23403       0.949922
14408       0.951487
3037        0.951487
        POS  position_prob
5997   5431       0.001565
5534  26678       0.001565
5533  28699       0.001565
5532  29000       0.001565
5531  28713       0.001565
       POS  position_prob
1287  2416       0.392801
1288  2416       0.392801
1289  2416       0.392801
1276  2416       0.392801
1448  2416       0.392801
Number of rules: 22
Boston dataset association rules: 
    antecedents consequents                                                    readable_rule  antecedent support  consequent support   support  confidence       lift  leverage  conviction
25     (953511)   (1375531)     ((9535, Silent, AF < 0.20)) => ((13755, Misense, AF < 0.20))            0.06

In [117]:
# Test get_position_probability()

pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Read the input file and pick the needed columns
in_file = "https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz"
df_in = pd.read_csv(in_file, sep='\t')[['Sample', 'POS', 'AF', 'FUNCLASS']]

position=21
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))

position=30
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))

position=29806
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))


num_samples: 639
num_positions: 1013
position: 21, position_prob: 0.0016
num_samples: 639
num_positions: 1013
position: 30, position_prob: -1.0000
num_samples: 639
num_positions: 1013
position: 29806, position_prob: 0.0078


---
**MBA parameter**s 

1.   **bos_rules**: *support*=0.223, *confidence*=0.905, *lift*=2.863, *conviction*=7.000
2.   **uke_rules**: *support*=0.210, *confidence*=0.910, *lift*=2.500, conviction *italicized text*=7.000
3.   **ukl_rules**: *support*=0.203, *confidence*=0.926, *lift*=2.030, *conviction*=7.000
---
The last digit of an entry is Allele Frequency (**AF**)

*   **3**: $>=$ 0.80    
*   **2**: $>=$ 0.20 and $<$ 0.80
*   **1**: $<$ 0.20
---
The digit before the last is **Funclass**

*  **4**: None
*  **3**: Missense
*  **2**: Nonsense
*  **1**: Silent
---
**Boston association rules** (12 rules)

**Antecedent** entries

* 3037**13**  (5 times)
* 14408**33** (5 times)
* 23403**33** (5 times)
* 26542**31** (All 12)

**Consequent** entries

* 3037**13** (5 times)
* 14408**33** (5 times)
* 23403**33** (5 times)
* 26545**31** (All 12)








---
**UK early association rules** (10 rules)

**Antecedent** entries

* 241**43**   (9 times)
* 14408**33** (3 times)
* 23403**33** (4 times)
* 28881**32** (All 10)

**Consequent** entries

* 3037**12**  (All 10)
* 14408**33** (3 times)
* 23403**33** (3 times)
* 28883**33** (All 10)

---
**UK late association rules** (11 rules)

**Antecedent** entries

* 204**42** (All 11)
* 445**13** (All 11)
* 6286**13** (3 times)
* 21255**13** (All 11)
* 23403**33** (3 times)
* 28932**32** (1 time)

**Consequent** entries

* 6286**13** (5 times)
* 22227**32** (All 11)
* 23403**33** (5 times)
* 27944**13** (10 times)
* 28932**32** (9 time)

---

**Entries that show up in antecedent/consequent across samples**

**Antecedent**

* 14408**33** (Boston, UKE)
* 23403**33** (Boston, UKE, UKL)

**Consequent**

* 3037**13** (Boston, UKE)
* 14408**33** (Boston, UKE)
* 23403**33** (Boston, UKE, UKL)

